In [7]:
using TuringModels

Turing.setadbackend(:reverse_diff)
Turing.turnprogress(false)

d = CSV.read(rel_path("..", "data", "Kline.csv"), delim=';');
size(d) # Should be 10x5

┌ Info: [Turing]: global PROGRESS is set as false
└ @ Turing /Users/rob/.julia/packages/Turing/r03H1/src/Turing.jl:24


(10, 5)

New col log_pop, set log() for population data

In [8]:
d[:log_pop] = map((x) -> log(x), d[:population]);
d[:society] = 1:10;

Turing model

In [9]:
@model m12_6(total_tools, log_pop, society) = begin
    # Total num of y
    N = length(total_tools)
    # priors
    α ~ Normal(0, 10)
    βp ~ Normal(0, 1)
    # Separate σ priors for each society
    σ_society ~ Truncated(Cauchy(0, 1), 0, Inf)
    # Number of unique societies in the data set
    N_society = length(unique(society)) #10
    # Vector of societies (1,..,10) which we'll set priors on
    α_society = Vector{Real}(undef, N_society)
    # For each society [1,..,10] set a prior N(0, σ_society)
    α_society ~ [Normal(0, σ_society)]

    for i ∈ 1:N
        λ = exp(α + α_society[society[i]] + βp*log_pop[i])
        total_tools[i] ~ Poisson(λ)
    end
end

m12_6 (generic function with 4 methods)

Sample

In [10]:
posterior = sample(m12_6(d[:total_tools], d[:log_pop],
    d[:society]), Turing.NUTS(4000, 1000, 0.95));
# Fix the inclusion of adaptation samples
draws = 1001:4000
posterior2 = Chains(posterior[draws,:,:], :parameters)

┌ Info: [Turing] looking for good initial eps...
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/r03H1/src/inference/support/hmc_core.jl:240
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/r03H1/src/core/ad.jl:169
┌ Warning: grad = [NaN, NaN, 3.975875826723833e81, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/r03H1/src/core/ad.jl:170
┌ Warning: Numerical error has been found in gradients.
└ @ Turing.Core /Users/rob/.julia/packages/Turing/r03H1/src/core/ad.jl:169
┌ Warning: grad = [NaN, NaN, 1.5404851023330582e36, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN, NaN]
└ @ Turing.Core /Users/rob/.julia/packages/Turing/r03H1/src/core/ad.jl:170
[NUTS{Turing.Core.FluxTrackerAD,Union{}}] found initial ϵ: 0.0125
└ @ Turing.Inference /Users/rob/.julia/packages/Turing/r03H1/src/inference/support/hmc_core.jl:235
┌ Info:  Adapted ϵ = 0.004603497304111276, std = [1.0, 1.0, 1.0, 1.0, 1.

[NUTS] Finished with
  Running time        = 196.16798833699977;
  #lf / sample        = 0.0;
  #evals / sample     = 189.32975;
  pre-cond. metric    = [1.0, 1.0, 1.0, 1.0, 1.0, 1.0,....


Object of type Chains, with data of type 3000×13×1 Array{Union{Missing, Float64},3}

Log evidence      = 0.0
Iterations        = 1001:4000
Thinning interval = 1
Chains            = Chain1
Samples per chain = 3000
parameters        = α_society[5], α_society[9], α_society[3], α_society[6], α_society[8], α_society[1], βp, α_society[10], σ_society, α, α_society[2], α_society[4], α_society[7]

parameters
                Mean    SD   Naive SE  MCSE     ESS  
            α  1.0131 0.8098   0.0148 0.1170  47.9312
 α_society[1] -0.1869 0.2628   0.0048 0.0274  91.8357
 α_society[2]  0.0725 0.2366   0.0043 0.0222 113.4874
 α_society[3] -0.0192 0.2089   0.0038 0.0160 171.3097
 α_society[4]  0.3666 0.2069   0.0038 0.0180 131.4825
 α_society[5]  0.0727 0.1925   0.0035 0.0126 232.8379
 α_society[6] -0.3322 0.2107   0.0038 0.0131 258.9057
 α_society[7]  0.1750 0.1813   0.0033 0.0110 273.2586
 α_society[8] -0.1702 0.1895   0.0035 0.0089 455.1627
 α_society[9]  0.2971 0.1775   0.0032 0.0089 397.5320
α_s

Results rethinking

In [11]:
m126rethinking = "
              Mean StdDev lower 0.89 upper 0.89 n_eff Rhat
a              1.11   0.75      -0.05       2.24  1256    1
bp             0.26   0.08       0.13       0.38  1276    1
a_society[1]  -0.20   0.24      -0.57       0.16  2389    1
a_society[2]   0.04   0.21      -0.29       0.38  2220    1
a_society[3]  -0.05   0.19      -0.36       0.25  3018    1
a_society[4]   0.32   0.18       0.01       0.60  2153    1
a_society[5]   0.04   0.18      -0.22       0.33  3196    1
a_society[6]  -0.32   0.21      -0.62       0.02  2574    1
a_society[7]   0.14   0.17      -0.13       0.40  2751    1
a_society[8]  -0.18   0.19      -0.46       0.12  2952    1
a_society[9]   0.27   0.17      -0.02       0.52  2540    1
a_society[10] -0.10   0.30      -0.52       0.37  1433    1
sigma_society  0.31   0.13       0.11       0.47  1345    1
";

Describe the posterior samples

In [12]:
describe(posterior2)

cnames = [
  :α,:α_society_1, :α_society_2, :α_society_3,
  :α_society_4, :α_society_5, :α_society_6,
  :α_society_7, :α_society_8,:α_society_9,
  :α_society_10, :βp, :σ_society
]

df = DataFrame(convert(Matrix{Float64}, to_df(posterior2)), cnames)
first(df, 5)

Log evidence      = 0.0
Iterations        = 1001:4000
Thinning interval = 1
Chains            = Chain1
Samples per chain = 3000
parameters        = α_society[5], α_society[9], α_society[3], α_society[6], α_society[8], α_society[1], βp, α_society[10], σ_society, α, α_society[2], α_society[4], α_society[7]



┌ Warning: `quantile(v::AbstractArray{<:Real})` is deprecated, use `quantile(v, [0.0, 0.25, 0.5, 0.75, 1.0])` instead.
│   caller = (::getfield(MCMCChains, Symbol("##102#104")){Chains{Union{Missing, Float64},Float64,NamedTuple{(:parameters,),Tuple{Array{String,1}}},NamedTuple{(:hashedsummary,),Tuple{Base.RefValue{Tuple{UInt64,MCMCChains.ChainSummaries}}}}}})(::String) at none:0
└ @ MCMCChains ./none:0


Empirical Posterior Estimates
─────────────────────────────────────────────────────
parameters
                Mean    SD   Naive SE  MCSE     ESS  
            α  1.0131 0.8098   0.0148 0.1170  47.9312
 α_society[1] -0.1869 0.2628   0.0048 0.0274  91.8357
 α_society[2]  0.0725 0.2366   0.0043 0.0222 113.4874
 α_society[3] -0.0192 0.2089   0.0038 0.0160 171.3097
 α_society[4]  0.3666 0.2069   0.0038 0.0180 131.4825
 α_society[5]  0.0727 0.1925   0.0035 0.0126 232.8379
 α_society[6] -0.3322 0.2107   0.0038 0.0131 258.9057
 α_society[7]  0.1750 0.1813   0.0033 0.0110 273.2586
 α_society[8] -0.1702 0.1895   0.0035 0.0089 455.1627
 α_society[9]  0.2971 0.1775   0.0032 0.0089 397.5320
α_society[10] -0.1077 0.2929   0.0053 0.0340  74.0449
           βp  0.2685 0.0857   0.0016 0.0122  49.1601
    σ_society  0.3309 0.1184   0.0022 0.0120  97.7657

Quantiles
─────────────────────────────────────────────────────
parameters
                2.5%   25.0%   50.0%   75.0%   97.5%
            α -1.909

,α,α_society_1,α_society_2,α_society_3,α_society_4,α_society_5,α_society_6,α_society_7,α_society_8,α_society_9,α_society_10,βp,σ_society
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.468659,0.0997899,0.0627841,0.131921,0.18443,0.0221748,-0.117291,0.189045,-0.257996,0.203787,-0.218552,0.319401,0.217443
2,0.586306,0.0517588,0.0721921,0.12386,0.126687,0.00442691,-0.0824298,0.174092,-0.202191,0.21546,-0.178376,0.312929,0.209458
3,0.687747,0.0621691,0.160311,-0.352538,0.432242,0.0729015,-0.32996,0.118331,-0.343966,0.29101,-0.123686,0.304788,0.194923
4,0.713992,0.0974287,0.22961,0.167159,0.280534,0.159729,-0.141922,0.0661826,-0.0955419,-0.0728205,-0.0705672,0.306378,0.179783
5,0.774082,0.13936,0.256276,0.129645,0.285317,-0.0341815,-0.220036,0.219055,0.0609183,0.0107175,-0.079961,0.28998,0.146416


End of m12.6t.jl

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*